In [ ]:
# Untuk file XML
!mkdir xml
!cd xml

!pip install roboflow

!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="UQ3LZ0w4sZMK52Ocd4hm")
project = rf.workspace("bangkit-7pvwz").project("foodies-qbezq")
version = project.version(3)
dataset = version.download("voc")


mkdir: cannot create directory ‘xml’: File exists
loading Roboflow workspace...
loading Roboflow project...


In [ ]:
%%bash
cd /content
git clone https://github.com/Cartucho/mAP /content/mAP
cd /content/mAP
rm input/detection-results/*
rm input/ground-truth/*
rm input/images-optional/*
wget https://raw.githubusercontent.com/EdjeElectronics/TensorFlow-Lite-Object-Detection-on-Android-and-Raspberry-Pi/master/util_scripts/calculate_map_cartucho.py

fatal: destination path '/content/mAP' already exists and is not an empty directory.
rm: cannot remove 'input/detection-results/*': No such file or directory
rm: cannot remove 'input/ground-truth/backup': Is a directory
--2024-07-21 15:07:22--  https://raw.githubusercontent.com/EdjeElectronics/TensorFlow-Lite-Object-Detection-on-Android-and-Raspberry-Pi/master/util_scripts/calculate_map_cartucho.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5397 (5.3K) [text/plain]
Saving to: ‘calculate_map_cartucho.py.1’

     0K .....                                                 100% 32.6M=0s

2024-07-21 15:07:22 (32.6 MB/s) - ‘calculate_map_cartucho.py.1’ saved [5397/5397]



In [ ]:
!cp /content/Foodies-3/test/* /content/mAP/input/images-optional # Copy images and xml files
!mv /content/mAP/input/images-optional/*.xml /content/mAP/input/ground-truth  # Move xml files to the appropriate folder

In [ ]:
!python /content/mAP/scripts/extra/convert_gt_xml.py

Conversion completed!


In [ ]:
!cp /content/Foodies-3/test/*.jpg /content/xml

In [ ]:
#!unzip detect

In [ ]:
# Import packages
import os
import cv2
import numpy as np
import sys
import glob
import random
import importlib.util
from tensorflow.lite.python.interpreter import Interpreter

import matplotlib
import matplotlib.pyplot as plt

%matplotlib inline

### Define function for inferencing with TFLite model and displaying results

def tflite_detect_images(modelpath, imgpath, lblpath, min_conf=0.5, num_test_images=10, savepath='/content/results', txt_only=False):

  # Grab filenames of all images in test folder
  images = glob.glob(imgpath + '/*.jpg') + glob.glob(imgpath + '/*.JPG') + glob.glob(imgpath + '/*.png') + glob.glob(imgpath + '/*.bmp')

  # Load the label map into memory
  with open(lblpath, 'r') as f:
      labels = [line.strip() for line in f.readlines()]

  # Load the Tensorflow Lite model into memory
  interpreter = Interpreter(model_path=modelpath)
  interpreter.allocate_tensors()

  # Get model details
  input_details = interpreter.get_input_details()
  output_details = interpreter.get_output_details()
  height = input_details[0]['shape'][1]
  width = input_details[0]['shape'][2]

  float_input = (input_details[0]['dtype'] == np.float32)

  input_mean = 127.5
  input_std = 127.5

  # Randomly select test images
  images_to_test = random.sample(images, num_test_images)

  # Loop over every image and perform detection
  for image_path in images_to_test:

      # Load image and resize to expected shape [1xHxWx3]
      image = cv2.imread(image_path)
      image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
      imH, imW, _ = image.shape
      image_resized = cv2.resize(image_rgb, (width, height))
      input_data = np.expand_dims(image_resized, axis=0)

      # Normalize pixel values if using a floating model (i.e. if model is non-quantized)
      if float_input:
          input_data = (np.float32(input_data) - input_mean) / input_std

      # Perform the actual detection by running the model with the image as input
      interpreter.set_tensor(input_details[0]['index'], input_data)
      interpreter.invoke()

      # Retrieve detection results
      boxes = interpreter.get_tensor(output_details[1]['index'])[0] # Bounding box coordinates of detected objects
      classes = interpreter.get_tensor(output_details[3]['index'])[0] # Class index of detected objects
      scores = interpreter.get_tensor(output_details[0]['index'])[0] # Confidence of detected objects

      detections = []

      # Loop over all detections and draw detection box if confidence is above minimum threshold
      for i in range(len(scores)):
          if ((scores[i] > min_conf) and (scores[i] <= 1.0)):

              # Get bounding box coordinates and draw box
              # Interpreter can return coordinates that are outside of image dimensions, need to force them to be within image using max() and min()
              ymin = int(max(1, (boxes[i][0] * imH)))
              xmin = int(max(1, (boxes[i][1] * imW)))
              ymax = int(min(imH, (boxes[i][2] * imH)))
              xmax = int(min(imW, (boxes[i][3] * imW)))

              cv2.rectangle(image, (xmin, ymin), (xmax, ymax), (10, 255, 0), 2)

              # Draw label
              object_name = labels[int(classes[i])] # Look up object name from "labels" array using class index
              label = '%s: %d%%' % (object_name, int(scores[i] * 100)) # Example: 'person: 72%'
              labelSize, baseLine = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.7, 2) # Get font size
              label_ymin = max(ymin, labelSize[1] + 10) # Make sure not to draw label too close to top of window
              cv2.rectangle(image, (xmin, label_ymin - labelSize[1] - 10), (xmin + labelSize[0], label_ymin + baseLine - 10), (255, 255, 255), cv2.FILLED) # Draw white box to put label text in
              cv2.putText(image, label, (xmin, label_ymin - 7), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 0), 2) # Draw label text

              # Print detection results
              print(f'Detected object: {object_name}, Confidence: {int(scores[i] * 100)}%')

              detections.append([object_name, scores[i], xmin, ymin, xmax, ymax])

      # All the results have been drawn on the image, now display the image
      if txt_only == False: # "text_only" controls whether we want to display the image results or just save them in .txt files
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        plt.figure(figsize=(12, 16))
        plt.imshow(image)
        plt.show()

      # Save detection results in .txt files (for calculating mAP)
      elif txt_only == True:

        # Get filenames and paths
        image_fn = os.path.basename(image_path)
        base_fn, ext = os.path.splitext(image_fn)
        txt_result_fn = base_fn + '.txt'
        txt_savepath = os.path.join(savepath, txt_result_fn)

        # Write results to text file
        # (Using format defined by https://github.com/Cartucho/mAP, which will make it easy to calculate mAP)
        with open(txt_savepath, 'w') as f:
            for detection in detections:
                f.write('%s %.4f %d %d %d %d\n' % (detection[0], detection[1], detection[2], detection[3], detection[4], detection[5]))

  return

In [ ]:
# Set up variables for running inference, this time to get detection results saved as .txt files
PATH_TO_IMAGES='/content/Foodies-3/test'   # Path to test images folder
PATH_TO_MODEL='/content/detect.tflite'   # Path to .tflite model file
PATH_TO_LABELS='/content/labelmap.txt'   # Path to labelmap.txt file
PATH_TO_RESULTS='/content/mAP/input/detection-results' # Folder to save detection results in
min_conf_threshold=0.1   # Confidence threshold

# Use all the images in the test folder
image_list = glob.glob(PATH_TO_IMAGES + '/*.jpg') + glob.glob(PATH_TO_IMAGES + '/*.JPG') + glob.glob(PATH_TO_IMAGES + '/*.png') + glob.glob(PATH_TO_IMAGES + '/*.bmp')
images_to_test = min(500, len(image_list)) # If there are more than 500 images in the folder, just use 500

# Tell function to just save results and not display images
txt_only = True

# Run inferencing function!
print('Starting inference on %d images...' % images_to_test)
tflite_detect_images(PATH_TO_MODEL, PATH_TO_IMAGES, PATH_TO_LABELS, min_conf_threshold, images_to_test, PATH_TO_RESULTS, txt_only)
print('Finished inferencing!')

Starting inference on 109 images...
Detected object: Nasi, Confidence: 97%
Detected object: Sayur, Confidence: 95%
Detected object: Tempe, Confidence: 94%
Detected object: Tempe, Confidence: 93%
Detected object: Tahu, Confidence: 89%
Detected object: Tempe, Confidence: 38%
Detected object: Nasi, Confidence: 94%
Detected object: Ikan, Confidence: 88%
Detected object: Tahu, Confidence: 85%
Detected object: Sayur, Confidence: 84%
Detected object: Ikan, Confidence: 18%
Detected object: Tahu, Confidence: 15%
Detected object: Ikan, Confidence: 15%
Detected object: Ikan, Confidence: 13%
Detected object: Tahu, Confidence: 13%
Detected object: Ikan, Confidence: 13%
Detected object: Nasi, Confidence: 86%
Detected object: Ikan, Confidence: 84%
Detected object: Telur, Confidence: 83%
Detected object: Telur, Confidence: 70%
Detected object: Sayur, Confidence: 61%
Detected object: Tempe, Confidence: 20%
Detected object: Telur, Confidence: 19%
Detected object: Telur, Confidence: 18%
Detected object: 

In [ ]:
%cd /content/mAP
!python calculate_map_cartucho.py --labels=/content/labelmap.txt

/content/mAP
Calculating mAP at 0.50 IoU threshold...
95.00% = Ayam AP 
100.00% = Daging AP 
91.68% = Ikan AP 
89.05% = Nasi AP 
91.72% = Sayur AP 
93.41% = Tahu AP 
96.53% = Telur AP 
92.76% = Tempe AP 
mAP = 93.77%
Calculating mAP at 0.55 IoU threshold...
95.00% = Ayam AP 
100.00% = Daging AP 
91.68% = Ikan AP 
89.05% = Nasi AP 
91.72% = Sayur AP 
93.41% = Tahu AP 
96.53% = Telur AP 
92.76% = Tempe AP 
mAP = 93.77%
Calculating mAP at 0.60 IoU threshold...
95.00% = Ayam AP 
100.00% = Daging AP 
91.68% = Ikan AP 
89.05% = Nasi AP 
89.77% = Sayur AP 
93.41% = Tahu AP 
96.53% = Telur AP 
91.64% = Tempe AP 
mAP = 93.39%
Calculating mAP at 0.65 IoU threshold...
95.00% = Ayam AP 
100.00% = Daging AP 
91.68% = Ikan AP 
89.05% = Nasi AP 
87.95% = Sayur AP 
93.41% = Tahu AP 
96.53% = Telur AP 
91.64% = Tempe AP 
mAP = 93.16%
Calculating mAP at 0.70 IoU threshold...
95.00% = Ayam AP 
100.00% = Daging AP 
88.55% = Ikan AP 
89.05% = Nasi AP 
84.33% = Sayur AP 
93.41% = Tahu AP 
96.53% = Telur AP 